In [ ]:
from trustifai import Trustifai, MetricContext
from langchain_core.documents import Document
from IPython.display import display, Markdown

In [ ]:
generate_engine = Trustifai(config_path="../config_file.yaml")

In [12]:
query = "What is Acme Corp's policy on remote work?"
answer = generate_engine.generate(prompt=query)

In [13]:
answer

{'response': 'I don’t have access to Acme Corp’s specific internal policies. However, if you are an employee or have access to their resources, you can usually find information about remote work policies in the company’s employee handbook, intranet, or by contacting Human Resources.\n\nIf you provide more context or specify the industry or location, I can offer general guidance on typical remote work policies. Let me know how you’d like to proceed!',
 'metadata': {'confidence_score': 0.6,
  'confidence_label': 'Low Confidence',
  'confidence_details': {'explanation': 'Model is uncertain about its output.',
   'avg_logprob': -0.34,
   'variance': 0.16,
   'token_count': 86},
  'logprobs_available': True}}

In [14]:
documents = [
    Document(
        page_content="Acme Corp announced a hybrid work model in 2023, requiring employees to be in office 3 days per week.",
        metadata={"source": "hr_policy_2023.pdf"}
    ),
    Document(
        page_content="The company provides home office stipends of up to $500 for remote setup.",
        metadata={"source": "benefits_guide.pdf"}
    )
]

metric_context = MetricContext(
    query=query,
    answer=answer['response'],
    documents=documents,
)
trust_engine = Trustifai(metric_context, "../config_file.yaml")

In [ ]:
trust_score = trust_engine.get_trust_score()
trust_score

{'score': 0.42,
 'label': 'UNRELIABLE',
 'details': {'evidence_coverage': {'score': 0.0,
   'label': 'Likely Hallucinated Answer',
   'details': {'explanation': 'Many claims lack support from source documents.',
    'strategy': 'LLM',
    'total_sentences': 4,
    'supported_sentences': 0,
    'unsupported_sentences': ['I don’t have access to Acme Corp’s specific internal policies.',
     'However, if you are an employee or have access to their resources, you can usually find information about remote work policies in the company’s employee handbook, intranet, or by contacting Human Resources.',
     'If you provide more context or specify the industry or location, I can offer general guidance on typical remote work policies.',
     'Let me know how you’d like to proceed!'],
    'failed_checks': 0}},
  'semantic_drift': {'score': 0.55,
   'label': 'Likely Hallucinated Answer',
   'details': {'explanation': 'Answer diverges significantly from source documents.',
    'total_documents': 2}

In [ ]:
reasoning_graph = trust_engine.build_reasoning_graph(trust_score)
mermaid_graph = trust_engine.visualize(reasoning_graph, graph_type='mermaid')
pyvis_graph = trust_engine.visualize(reasoning_graph, graph_type='pyvis') #exported to html file

In [25]:
display(Markdown(mermaid_graph))

```mermaid
flowchart TD
   evidence_coverage["<b>Evidence Coverage</b><br/>Score: 0.00<br/>Likely Hallucinated Answer"]
   semantic_drift["<b>Semantic Drift</b><br/>Score: 0.55<br/>Likely Hallucinated Answer"]
   consistency["<b>Consistency</b><br/>Score: 0.86<br/>Stable Consistency"]
   source_diversity["<b>Source Diversity</b><br/>Score: 0.85<br/>High Trust"]
   trust_aggregation{"<b>Trust Score</b><br/>Score: 0.42"}
   final_decision("<b>Decision: UNRELIABLE</b>")
    evidence_coverage --> trust_aggregation
    semantic_drift --> trust_aggregation
    consistency --> trust_aggregation
    source_diversity --> trust_aggregation
    trust_aggregation --> final_decision
    style evidence_coverage fill:#ff6b6b,color:#000000
    style semantic_drift fill:#ff6b6b,color:#000000
    style consistency fill:#2ecc71,color:#000000
    style source_diversity fill:#2ecc71,color:#000000
    style trust_aggregation fill:#ff6b6b,color:#000000
    style final_decision fill:#ff6b6b,color:#000000
```